### Para este proyecto se ocupo la librería de Spotify para consumir recursos de la API y poder hacer las peticiones correctamente.

In [4]:
from spotipy.oauth2 import SpotifyClientCredentials, SpotifyOAuth # type: ignore
import spotipy # type: ignore
import sys
from pprint import pprint
from pymonad.tools import curry
from pymonad.reader import Compose
import pymonad.tools
from oslash import Just, Nothing, List
import functools
import operator

In [5]:
client_id = "01f2938c5262491da9b4ccfc11aea077"
client_secret = "978c8f2e99f84ad99a7f9f13ac59a3fa"
sp = spotipy.Spotify(client_credentials_manager = SpotifyClientCredentials(client_id, client_secret))

#### 1.- Obtener 20 canciones de un artista dado por el usuario.

In [6]:
def generador(author:str):
    result = sp.search(author, limit=20)
    for i in range(len(result["tracks"]["items"])):
        yield result["tracks"]["items"][i]["name"]

if __name__ == '__main__':
    author = input("Ingrese el nombre del autor: ")
    for i,j in enumerate(generador(author), start=1):
        print(str(i)+": "+j)

1: Ahora Te Puedes Marchar
2: La Media Vuelta
3: La Incondicional
4: Culpable O No - Miénteme Como Siempre
5: Sabor a Mi
6: Hasta Que Me Olvides
7: Cuando Calienta El Sol
8: No Sé Tú
9: Por Debajo De La Mesa
10: La Mentira
11: El Ultimo Adiós - Varios Artistas Version
12: Entrégate
13: La bikina - En vivo
14: Inolvidable
15: Tengo Todo Excepto a Ti
16: Suave
17: Si Nos Dejan - En Vivo
18: Usted
19: Fría Como El Viento
20: Sabes Una Cosa


#### 2.- Obtener la cancion mas larga de las primeras 20 canciones de un artista dado por el usuario.

In [7]:
def oneover(x):
    try:
        ret = x[1]/60000
        print("La cancion mas larga encontrada es: ",x[0])
    except:
        return Nothing()
    return Just(ret)

def leer(autor:str):
    result = sp.search(autor, limit=20)
    listaAux = list(map(lambda x :[x["name"], x["duration_ms"]], result["tracks"]["items"]))
    def maxL():
        aux2 = max(listaAux, key=lambda x : x[1])
        a = Just(aux2).bind(oneover)
        print("Con una duracion de: ",a)
    return maxL

if __name__ == '__main__':
    author = input("Ingrese el nombre del autor: ")
    r = leer(author)
    r()

La cancion mas larga encontrada es:  Suave
Con una duracion de:  Just 4.826666666666667


#### 3.- Crea una tupla de tuplas con los generos: banda, rock, electronica y 20 canciones de cada genero.

In [8]:
def generador(genre:str)->list:
    result = sp.search(q = 'genre' + genre, limit = 20)
    return list(map(lambda x : x["name"], result["tracks"]["items"]))

if __name__ == '__main__':
    generos= ["Banda", "Rock", "Electronica"]
    canciones = list(map(lambda x : generador(x), generos))
    final= tuple(zip(generos, canciones))
    print(final)

(('Banda', ['Cuál Adiós', 'BANDS!', 'El Gato y Yo', 'Bands', 'Banda', 'Banda', 'El Cabrón', 'No Me Quedaron Ganas', 'Alaba', 'Bands', 'No Me Perdones', 'Déjame/Ya No Vives En Mí (Cuál Adiós) - En Vivo', 'Bands', 'Negocio Cuajado/El Quesito - En Vivo', 'Dime Cómo', 'Bands', 'BANDO', 'Ingrato Querer', 'Yo Soy Generasion', 'Paixão Bandida']), ('Rock', ['Rockstar', 'Paradise City', 'Spiders', 'Rock Of Ages', 'Rock Me', 'Rocket Queen', 'Rock Me Amadeus', 'Surfacing', 'Surfacing', 'Rock Me', 'Rock Rock (Till You Drop)', 'Brompton Cocktail', "Rock 'n' Roll High School - 2002 Remaster", 'Rock Me on the Water', 'Deutschland', 'Rock and Roll - Remaster', 'Rock On', 'Rockstar', 'Rock My Heart - Radio Mix', "Sweet Child O' Mine"]), ('Electronica', ['Música Electrónica Nueva Del 2020 #8', 'ELECTRÓNICA', 'Electronica (Guaracha 2022)', 'Electrónica Con Mucho Bajo Par Retumbar', 'Music Metaverse', 'Música Electrónica Nueva Del 2022 #7', 'Música Electrónica Nueva Del 2021 #41', 'Música Electrónica Nuev

#### 4.- Obten el tiempo total de las primeras 30 canciones de un artista dado por el usuario.

In [9]:
def fun(author:str)->list:
    result = sp.search(author, limit=30)
    aux2= []
    return list(map(lambda x: x["duration_ms"], result["tracks"]["items"]))

if __name__ == '__main__':
    author = input("Ingrese el nombre del autor: ")
    r = fun(author)
    r2= functools.reduce(operator.add,r)/ 60000
    final = Just(r2)
    print(final)

Just 124.2601


#### 5.- Obtener 10 canciones de un artista dado por el usuario.

In [10]:
def read_artist()->str:
    l = input('Ingrese un artista: ')
    return l

def search1(la:str)-> list:
    b = sp.search(la, limit=10)
    return list(map(lambda x: x["name"], b["tracks"]["items"]))
    
if __name__ == '__main__':
    print(search1(read_artist()))

['Ahora Te Puedes Marchar', 'La Media Vuelta', 'La Incondicional', 'Culpable O No - Miénteme Como Siempre', 'Sabor a Mi', 'Hasta Que Me Olvides', 'Cuando Calienta El Sol', 'No Sé Tú', 'Por Debajo De La Mesa', 'La Mentira']


#### 6.- Crear un conjunto donde las canciones del top 10 de luis miguel no esten en las canciones generales de ese artista

In [11]:
def dif(a:str, u:str)-> list:
    b = sp.search(a, limit=10)
    lista = list(map(lambda x: [0, x["name"]], b["tracks"]["items"]))
    
    response = sp.artist_top_tracks(u)
    lista += list(map(lambda x: [1, x["name"]], response["tracks"]))
    return lista

if __name__ == '__main__':
    urn = 'spotify:artist:2nszmSgqreHSdJA3zWPyrW'
    c1 = set()
    c2 = set()
    lista2 = dif('Luis Miguel', urn)
    c1 = {x[1] for x in filter(lambda x: x[0] == 0, lista2)}
    c2 = {x[1] for x in filter(lambda x: x[0] != 0, lista2)}
    # print(c1)
    # print(c2)
    c3 = c2.symmetric_difference(c2.intersection(c1))
    print(c3)

{'Sabor a Mi', 'Tengo Todo Excepto a Ti', 'La bikina - En vivo'}


#### 7.- Mostrar una lista de canciones del albúm "00:00" del artista "Siddhartha", pero eliminando la primera canción

In [19]:
def tail(x:list):
    if x:
        del x[0]

if __name__ == '__main__':
    urn = 'spotify:album:371qTqzTdc0reynwwfM2Bt'
    a = sp.album(urn)
    a1 = list(map(lambda x: x["name"], a["tracks"]["items"]))
    # print(a1)
    tail(a1)
    print(a1)

['Cardúmenes', 'Paraíso Lunar', '00:00', 'Huracanes', 'Mapa', '80 Días', 'Balsa', 'Suburbios', 'No Es Antes Ni Es Después']


#### 8.- Mostrar una lista con los géneros del artista "José José"

In [13]:
def closer(a):
    def genres(g):
        artist = list(map(lambda x : x, sp.artist(a)[g]))
        print(artist)
    return genres

if __name__ == '__main__':
    urn = 'spotify:artist:4mN0qcMxWX8oToqfDPM5yV'
    c = closer(urn)
    c('genres')

['bolero', 'cancion melodica', 'latin pop', 'ranchera']


#### 9.- Promedio de duración en ms de las primeras 30 canciones de un artista dado por el usuario.

In [14]:
@curry(1)
def sumc(x):
    try:
        return sum(x)
    except:
        return Nothing()

def oneover(x):
    try:
        ret = x[0]/x[1]
    except:
        return Nothing()
    return Just(ret)

def fun(author:str)->list:
    result = sp.search(author, limit=30)
    aux2 = list(map(lambda x: x["duration_ms"], result["tracks"]["items"]))
    return sumc(aux2), len(result["tracks"]["items"])

if __name__ == '__main__':
    g = input('Ingrese un artista: ')
    a =Just(fun(g)).bind(oneover)
    print(a)

Just 248520.2


#### 10.- Obtener los artistas de 50 canciones en general que colaboraron en un artista dado por el usuario.

In [15]:
def generador(author:str)->list:
    result = sp.search(author, limit=50)
    return list(map(lambda x: x["artists"], result["tracks"]["items"]))
    
def artistas(it:list)->set:
    aux = []
    for i in it:
        aux += list(map(lambda x : x["name"], i))
    return(set(aux))

if __name__ == '__main__':
    autor = input("Ingrese el nombre del autor: ")
    print(artistas(generador(autor)))


{'Romeo Santos', 'Shakira', 'Luis Fonsi', 'Wilkins', 'José Feliciano', 'La Ley', 'Gian Marco', 'Jennifer Lopez', 'Jon Secada', 'Christina Aguilera', 'Olga Tañón', 'Limi-T 21', 'Luis Miguel Del Amargue', 'Marco Antonio Solís', 'Ana Gabriel', 'Alicia Villarreal', 'Paulina Rubio', 'Ricardo Montaner', 'Chayanne', 'Lucía Méndez', 'Gloria Estefan', 'Yuri', 'Ricky Martin', 'Thalia', 'Miguel Bosé', 'Carlos Ponce', 'Luis Miguel', 'Gilberto Santa Rosa', 'Alejandro Fernández', 'Melina Leon', 'José José', 'Celia Cruz', 'Jaci Velasquez', 'Alberto "Beto" Zapata', 'Ana Bárbara', 'Dj Adoni', 'Alejandro Sanz', 'Emmanuele', 'Los Tigres Del Norte', 'Jorge Hernández', 'Juan Luis Guerra 4.40', 'Carlos Vives', 'José Luis Rodríguez', 'Giselle'}


#### 11.- Obtener un diccionario de diccionarios donde se muestre a los artistas con una lista de 30 de sus canciones

In [18]:
autores = []
dicc = {}
n = int(input('Ingrese el número de artistas: '))
for i in range (n):
    autores.append(input('Ingrese el nombre del artista: '))
tupla = [{autor:list(map(lambda x : x["name"], sp.search(autor, limit=30)["tracks"]["items"]))} for autor in autores]
for x, t in enumerate(tupla, start = 1):
    dicc['Cantante '+ str(x)] = t
print(dicc)


{'Cantante 1': {'luis miguel': ['Ahora Te Puedes Marchar', 'La Media Vuelta', 'La Incondicional', 'Culpable O No - Miénteme Como Siempre', 'Sabor a Mi', 'Hasta Que Me Olvides', 'Cuando Calienta El Sol', 'No Sé Tú', 'Por Debajo De La Mesa', 'La Mentira', 'El Ultimo Adiós - Varios Artistas Version', 'Entrégate', 'La bikina - En vivo', 'Inolvidable', 'Tengo Todo Excepto a Ti', 'Suave', 'Si Nos Dejan - En Vivo', 'Usted', 'Fría Como El Viento', 'Sabes Una Cosa', 'De Rodillas Te Pido', 'México En La Piel', 'No Me Platiques Más', 'La Barca', 'Amarte es un placer', 'Voy a Apagar La Luz / Contigo Aprendí', 'Medley (Yo Que No Vivo Sin Ti / Culpable O No / Más Allá De Todo / Fría Como El Viento / Entrégate /Tengo Todo Excepto A Ti / La Incondicional) - En Vivo', 'Mucho Corazón', 'Será Que No Me Amas (Blame It on the Boogie)', 'El Reloj']}, 'Cantante 2': {'jose jose ': ['El Amar y el Querer', 'Feliz Navidad', 'El Triste', 'Vamos a Darnos Tiempo', 'JGL', 'Almohada', 'Gavilán o Paloma', 'Hero (feat.